In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install einops
%pip install wandb

     |████████████████████████████████| 1.8MB 15.2MB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 133kB 53.9MB/s 
     |████████████████████████████████| 163kB 51.9MB/s 
     |████████████████████████████████| 71kB 12.0MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=fa7125ee31cff557325f8be757ab58802c9dde77396d9ef011e779d20743de5a
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=a26bebdbeeaa6d64e252269c6384bc6b8cc475a40215d90b3f2c78626806975c
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [4]:
%cd drive/MyDrive/Colab Notebooks/ViT

/content/drive/MyDrive/Colab Notebooks/ViT


In [5]:
from model.vit import ViT
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [6]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [7]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 64,
    "patch_size" : 16,
    "num_classes" : 10,
    "dim" : 128,
    "depth" : 12,
    "heads" : 8,
    "mlp_dim" : 1024,
    "channels" : 3,
    "dropout" : 0.,
    "emb_dropout": 0.
    }

# Training settings
import wandb
wandb.init(config=config, project="Cifar10_ViT", entity='NoguNogu',name='Vanila')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
trainloader, testloader = set_transforms(
    config['image_size'],
    config['batch_size'],
    name='Cifar10')._set_transforms()

In [9]:
model = ViT(
    dim = config['dim'],
    image_size = config['image_size'],
    patch_size = config['patch_size'],
    num_classes= config['num_classes'],
    channels= config['channels'],
    depth= config['depth'],
    heads= config['heads'],
    mlp_dim = config['mlp_dim']
    ).to(device)

In [10]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [11]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 2.0847 - acc: 0.2146 - val_loss : 2.0312 - val_acc: 0.2469




Epoch : 2 - loss : 1.9715 - acc: 0.2724 - val_loss : 1.9146 - val_acc: 0.2989




Epoch : 3 - loss : 1.8584 - acc: 0.3203 - val_loss : 1.7733 - val_acc: 0.3549




Epoch : 4 - loss : 1.7463 - acc: 0.3672 - val_loss : 1.7119 - val_acc: 0.3840




Epoch : 5 - loss : 1.6811 - acc: 0.3923 - val_loss : 1.6721 - val_acc: 0.3908



...


Epoch : 16 - loss : 1.3835 - acc: 0.5032 - val_loss : 1.4127 - val_acc: 0.4913




Epoch : 17 - loss : 1.3609 - acc: 0.5136 - val_loss : 1.3892 - val_acc: 0.5009




Epoch : 18 - loss : 1.3495 - acc: 0.5194 - val_loss : 1.3703 - val_acc: 0.5086




Epoch : 19 - loss : 1.3395 - acc: 0.5222 - val_loss : 1.3623 - val_acc: 0.5124




Epoch : 20 - loss : 1.3229 - acc: 0.5305 - val_loss : 1.3590 - val_acc: 0.5126



In [12]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun May  9 05:54:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    44W / 250W |   1311MiB / 16280MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------